README: <br>
- label propagated using kernel = 'rbf' (takes much shorter time than knn to train)

In [2]:
import pandas as pd
import os
os.chdir('/Users/liyuan/desktop/CSAir/codes')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import time
import json

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import confusion_matrix, classification_report

from semi_supervise import Semi_Supervise

In [3]:
ss = Semi_Supervise()
# load labeled data
labeled_data = ss.load_labeled_data('../res/labeled_data_with_without_tk.csv')
labeled_data.head()

# load unlabeled data
unlabeled_data = ss.load_unlabeled_data_csv('../res/unlabeled_review_5000.csv')
# concatenate labeled and unlabeled data
data_concat = ss.concat_data()
data_concat.head()

there are 1550 examples in labeled dataset


/Users/liyuan/Desktop/CSAir/codes/semi_supervise.py:53: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.data_concat = pd.concat([self.labeled_data, self.unlabeled_data],ignore_index=True)


,review,review_tokens,label,label_encoded
0,NaN,想 积分换 购 机票,N/A,NaN
1,机场值机，电子登机牌不能登录？？？,机场 值机 电子 登机牌 登录,出发,1.0
2,NaN,国际航班 选座 操作,N/A,NaN
3,订机票，查询航班首选南方航空，暑假赶紧带孩子出去旅游，释放心情！,订机票 查询 航班 首选 南方 航空 暑假 赶紧 孩子 旅游 释放 心情,预订,9.0
4,NaN,订 南航 官网 机票 电子客票 号 南航 官网 验真,N/A,NaN


In [4]:
# get X and y
X = ss.get_X().toarray()
y = data_concat.label_encoded.values

In [5]:
# checking
print('There are %d labeled review'%len(labeled_data))
print('There are %d reviews in total'%len(data_concat))

There are 1550 labeled review
There are 6426 reviews in total


In [20]:
n_total_samples = len(y)
n_labeled_points = len(labeled_data)

indices = np.arange(n_total_samples)
unlabeled_set = indices[n_labeled_points:]
print(unlabeled_set)

y_train = np.copy(y)
# convert np.nan to -1 => when feeding into label_propagation, cannot contain NA values
y_train[np.isnan(y_train)] = -1
y_train

start_time = time.time()
# try kernel = 'rbf' vs. kernel = 'knn'(treid)
lp_model = label_propagation.LabelSpreading(kernel = 'rbf')
lp_model.fit(X, y_train)
predicted_labels = lp_model.transduction_[unlabeled_set]
true_labels = y[unlabeled_set]
print('true labels: ', true_labels)
print('predicted labels:', predicted_labels)
print("--- %s seconds ---" % (time.time() - start_time))

[1550 1551 1552 ... 6423 6424 6425]
true labels:  [nan nan nan ... nan nan nan]
predicted labels: [0. 1. 9. ... 4. 3. 1.]
--- 11.003097772598267 seconds ---


In [24]:
# re-put prediction results to unlabeleda data df
unlabeled_data['labels_pred'] = predicted_labels

# load label index dictionary
with open('../res/labels_encoded_dict.txt','r') as f:
    labels_encoded_dict = json.load(f)
print(labels_encoded_dict)
    
def get_class(row):
    for k in list(labels_encoded_dict.keys()):
        if labels_encoded_dict[k] == row:
            return k
        
# match predicted labels to reviews
unlabeled_data['labels_pred'] = unlabeled_data['labels_pred'].apply(get_class)
unlabeled_data.head(10)

{'预订': 9, '出发': 1, '设计': 8, '性能': 4, '到达': 2, '行程': 6, '机上': 5, '计划': 7, '中转': 0, '售后': 3}


,reviews,review_tokens,label,label_encoded,labels_pred
0,查不到航班,查不到 航班,N/A,NaN,中转
1,我要取消在线选座的登机牌,我要 取消 在线 选座 登机牌,N/A,NaN,出发
2,本人被额外多办一张会员卡，南航会员办理流程是否需本人办理，而且我在被办理会员卡为郭忠能的时间...,额外 多办 一张 会员卡 南航 会员 办理 流程 需 办理 办理 会员卡 郭忠能 时间段 乘...,N/A,NaN,预订
3,打不開。半個小時依然打不開。浪費時間,打不開 半個 小時 依然 打不開 浪費 時間,N/A,NaN,出发
5,app上怎么也查不到还要飞多少里程或航段才能升级？,查不到 飞 里程 航段 升级,N/A,NaN,性能
6,我在广州白云机场办了南航的积分卡，现在暂时不准备去广东工作，卡不用寄到所写联系地址了，还有工...,广州白云机场 办 南航 积分 卡 暂时 广东 工作 卡 不用 寄到 写 联系地址 工作 单位...,N/A,NaN,性能
7,别人的信息怎么通知到我这里,信息 通知,N/A,NaN,出发
8,3月26日搭乘CZ8426从大阪飞深圳，遗留一把伞在飞机上，座位是35K，知道有没有看到，谢谢,月 日 搭乘 大阪 飞 深圳 遗留 一把 伞 飞机 座位 有没有 谢谢,N/A,NaN,性能
9,2018年05月17日已经反馈.....每次签到都要输入手机号，然后短信回复验证码才能签到，...,年 月 日 反馈 每次 签到 输入 手机号 短信 回复 验证码 签到 手机 营业厅 串通 赚...,N/A,NaN,售后
10,价格比别的网站贵一倍,价格比 网站 贵 一倍,N/A,NaN,售后


In [29]:
# result output to csv
output_df = unlabeled_data[['reviews','labels_pred']]
output_df.to_csv('../res/unlabeled_propagation_rbf.csv', index = False)